# Character Generation ✍

[Referencia](https://colab.research.google.com/drive/1ysEKrw_LE2jMndo1snrZUh5w87LQsCxk#forceEdit=true&sandboxMode=true&scrollTo=Fo3WY-e86zX2)

### 🎭Shakespeare play generator:

In [68]:
import keras
import tensorflow as tf
import os
import numpy as np
import time

## Datos:

### Importandolo de una web:

In [7]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

### Importandolo localmente:

In [8]:
# from google.colab import files
# path_to_file = list(files.upload().keys())[0]

### Leo los datos

In [9]:
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')

In [10]:
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



## Vectorizar el texto

In [43]:
# EXAMPLE
# example_texts = ['abcdefg', 'xyz']
# chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
# para unir caracteres y pasarlos a un array de str
# tf.strings.reduce_join(chars, axis=-1).numpy()

In [44]:
vocab = sorted(set(text))

### Encode

In [45]:
# convertir texto -> ids con la capa StringLookup
ids_from_chars = tf.keras.layers.StringLookup(
                    vocabulary=list(vocab),
                    mask_token=None)

all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))

### Decode

In [46]:
# convertir ids -> texto con la capa StringLookup
chars_from_ids = tf.keras.layers.StringLookup(
                    vocabulary=ids_from_chars.get_vocabulary(),
                    invert=True,
                    mask_token=None)

def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)


In [47]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

## Creacion de Training Examples

Remember our task is to feed the model a sequence and have it return to us the next character. This means we need to split our text data from above into many shorter sequences that we can pass to the model as training examples. 

The training examples we will prepapre will use a *seq_length* sequence as input and a *seq_length* sequence as the output where that sequence is the original sequence shifted one letter to the right. For example:

```input: Hell | output: ello```

Our first step will be to create a stream of characters from our text data.

In [48]:
seq_length = 100  # length of sequence for a training example
examples_per_epoch = len(text)//(seq_length+1)

sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

In [49]:
def split_input_target(chunk):  # for the example: hello
    input_text = chunk[:-1]  # hell
    target_text = chunk[1:]  # ello
    return input_text, target_text  # hell, ello

dataset = sequences.map(split_input_target)  # we use map to apply the above function to every entry

In [50]:
for input_example_batch, target_example_batch in dataset.take(1):
    print("Input :", text_from_ids(input_example_batch).numpy())
    print("Target:", text_from_ids(target_example_batch).numpy())

Input : b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target: b'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


In [51]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

## Creando el modelo

In [52]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [53]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [54]:
model = MyModel(
    # Be sure the vocabulary size matches the `StringLookup` layers.
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [58]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)

sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [59]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b"d we, in pity of the gentle king,\nHad slipp'd our claim until another age.\n\nGEORGE:\nBut when we saw "

Next Char Predictions:
 b"K,ipjrv?cMR,3[UNK]CaArwm\nwl:.3XDVD''lUSv:xPAGdOj emCz'KiCdW'YAYUXPgUyxbqPy[UNK]E lU\nmMvScz?zX N[UNK]rVAZJRlMigDI"


## Loss Function

In [60]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [61]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 66)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.190167, shape=(), dtype=float32)


## Compilo el modelo

In [62]:
model.compile(optimizer='adam', loss=loss)

## Train model

Antes de entrenarlo, vamos a hacer que el modelo guarde checkpoints a medida que se entrena, para poder cargar el modelo y reentrenarlo luego.

In [63]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [64]:
EPOCHS = 2 #20
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/2
172/172 [==============================] - 367s 2s/step - loss: 2.7095
Epoch 2/2
172/172 [==============================] - 439s 3s/step - loss: 1.9778


## Generando texto

### OneStep ~ de a un caracter

La forma más sencilla de generar texto con este modelo es ejecutarlo en un bucle y realizar un seguimiento del estado interno del modelo a medida que lo ejecuta.

In [65]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [66]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [84]:
def complete_text_ostep(n_char, input_text):
    next_char = tf.constant([input_text])
    states = None
    result = [next_char]

    for n in range(n_char):
        next_char, states = one_step_model.generate_one_step(next_char, states=states)
        result.append(next_char)

    tf_resp = tf.strings.join(result)
    result = tf_resp[0].numpy().decode('utf-8')
    return result

In [85]:
out = complete_text_ostep(1000, "ROMEO: ")
print(out)

ROMEO: my land,
Gase it a'l hant me corffricon;
Lot: this hengus and Noriming uppreessing.

GROMIO:
What I mo in Palls louse? 
Sich; wene's it waswands?

KONGY SIS:
Ther, when noving mant withel, which swis betans?

TRYorTIO:
She bugh say mein you reaved unong,
mad, and slese and brande ans me sould wrem.

Pecond,
My, soo home in this door Minot
Jut all the tell it frieeat of subupor, bload, sorem
to himbentlay ently paroBee
Than a doust with not but, in! Shy, a ceacu my?

LUKE:
Ny brisad 'tfore and thene then hized,
Thy gold fie stown, wher whan should liedd you to sevell youl stalu;

ADIY LOUG:
Whall me: wir you will fe aros for.
And trath the grue he pives mis owndirisoll onest hath,
And your good eets mant me the many beture's marr he wore
Is there ustrange ad must reather?-
RCONIZABEO:
That, to your kisband of hin
To prentur mespirde, all sit,
Thou nom paitese to daid to player farsht brivy beny'd,

LUCKIO:
Unlat, jear my lord to ie that hay in till mo!
Dape is tremins croods;
No,

### MultipleSteps ~ mas de un caracter a la vez

In [94]:
def complete_text_msteps(input_text):  
    
    # Output con tamaño fijo, 
    # Implementar

    states = None

    # Convert strings to token IDs.
    inputs = tf.constant([input_text])
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = model(inputs=input_ids,
                                     states=states,
                                     return_state=True)
    
    return 1

In [96]:
# complete_text_msteps("ROMEO: ")